<div class="alert alert-block alert-info">
     <b>Import Modules Needed</b> <i>(need pandas >= 1.2 and python >= 3.9.0 for all the data cleaning techniques to work)</i>
</div>

In [ ]:
import pandas as  pd
import os
import sweetviz as sv
import numpy as np
import dtale
import cudf
import dask_cudf

#!pip install git+https://github.com/innovationOUtside/fstring-magic.git

<div class="alert alert-block alert-info">
    <b>Jupyter Plumbing</b>
</div>


In [ ]:
# %load C:\users\derek\.jupyter\startup.py
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#pd.set_option('display.max_colwidth', 100)

pd.set_option('display.max_colwidth', None)



pd.set_option("display.max_rows", None)

pd.options.display.max_columns = 500

pd.options.display.max_seq_items = 500



<div class="alert alert-block alert-info">
    <b>Find CSV Files in working dir</b>
</div>

In [ ]:
csvStore = os.path.join(os.getcwd(),'__csvStore' )
csvlist = []
for file in enumerate(os.listdir(csvStore)):
    if file[1].endswith(".csv"):
        csvlist.append(os.path.join(str(csvStore), file[1]))
        print(os.path.join(str(csvStore), file[1]))

In [ ]:
list(enumerate(csvlist))

<div class="alert alert-block alert-info">
    <b>Import the raw data</b>
</div>

In [ ]:
df = pd.read_csv(csvlist[2])#.iloc[: , 2:]


In [ ]:
dtale.show(df)

<div class="alert alert-block alert-info">
    <b>Find Columns that contain mixed data types to investigate for possible data quality issues</b>
</div>


In [ ]:
if mixed_dtypes := {c: dtype for c in df.columns if (dtype := pd.api.types.infer_dtype(df[c])).startswith("mixed")}:
    raise TypeError(f"Dataframe has one more mixed dtypes: {mixed_dtypes}")

<div class="alert alert-block alert-info">
    <b>Find Columns with Dates and convert them to datetime type</b>
</div>

In [ ]:
cols_w_dates = df.columns[(df.columns.str.contains('DT', case=False) | df.columns.str.contains('date', case=False) | df.columns.str.contains('OFEPISODE', case=False))] 
df[cols_w_dates] = df[cols_w_dates].apply(pd.to_datetime)
cols_w_dates


<div class="alert alert-block alert-info">
    <b>Break up columns</b>
</div>

In [ ]:
# Descriptive IDs
df.columns[:27]
colsMeta = df.columns[:27]
# OASIS Factors
df.columns[27:]
colsOASIS = df.columns[27:]

<div class="alert alert-block alert-info">
    <b>Important Summary Stats</b>
</div>

In [ ]:
f"There are {df.groupby('DispositionId').agg({'pa_id': 'nunique'}).sum().values[0]} patients"

f"Alive: {df.groupby('mortality').agg({'pa_id': 'nunique'}).iloc[0,0]} | Dead: {df.groupby('mortality').agg({'pa_id': 'nunique'}).iloc[1,0]}"

<hr style="height:2px;border-width:0;color:black;background-color:black">

<div class="alert alert-info" role="alert">
    <b>Create a patients df and populate the death date by taking the most recent dates</b>
</div>

In [ ]:
len(df.groupby(['pa_id'])['pa_id','DispositionId','pa_gender','year_born', 'M0906_DC_TRAN_DTH_DT', 'mortality'].transform('max').drop_duplicates('pa_id'))

df_patients = df.groupby(['pa_id'])['pa_id', 'DispositionId' , 'pa_gender','year_born', 'M0906_DC_TRAN_DTH_DT', 'mortality', 'cedd_dateofdeath'].transform('max').drop_duplicates('pa_id').reset_index(drop=True)

<div class="alert alert-info" role="alert">
    <b>If no cedd_dateofdeath is provided then use the 'M0906_DC_TRAN_DTH_DT' </b>
</div>

In [ ]:
df_patients['cedd_dateofdeath'].where(((df_patients.mortality == 0) | (~df_patients.cedd_dateofdeath.isnull())), (df_patients['M0906_DC_TRAN_DTH_DT']), inplace=True)

<div class="alert alert-info" role="alert">
    <b>Create a Year of Death + Age at Death column</b>
</div>

In [ ]:
df_patients['yod']= pd.DatetimeIndex(df_patients['cedd_dateofdeath']).year
df_patients['aod'] = df_patients['yod'] - df_patients['year_born']


<div class="alert alert-info" role="alert">
    <b>Convert YOD and AAD into integers</b>
</div>

In [ ]:
#df_patients.fillna({'cedd_dateofdeath':0,}, inplace=True)
df_patients.fillna({'yod':0,}, inplace=True)
df_patients["yod"] = df_patients["yod"].apply(np.int64)
df_patients.fillna({'aod':0,}, inplace=True)
df_patients['aod'] = df_patients['aod'].apply(np.int64)

In [ ]:
dtale.show(df_patients)

<div class="alert alert-info" role="alert">
    <b>Export df_patients to .csv</b>
</div>

In [ ]:
df_patients.to_csv(csvStore+'/patients.csv', sep=',', encoding='utf-8', index=False)

<hr style="height:2px;border-width:0;color:black;background-color:black">

<div class="alert alert-warning" role="alert">
    <b>Find Best Columns to Use for Event Data </b>
</div>

In [ ]:
(df.loc[:,colsOASIS][df['mortality']==0].isnull().sum()/len(df.loc[:,colsOASIS][df['mortality']==0])).sort_values(ascending=False)
#mask2 = (df.loc[:,colsOASIS][df['DispositionId']==2].isnull().sum()/len(df.loc[:,colsOASIS][df['DispositionId']==2])>=.75).sort_values(ascending=False)

In [ ]:
(df.loc[:,colsOASIS][df['mortality']==1].isnull().sum()/len(df.loc[:,colsOASIS][df['mortality']==1])).sort_values(ascending=True)
#mask2 = (df.loc[:,colsOASIS][df['DispositionId']==2].isnull().sum()/len(df.loc[:,colsOASIS][df['DispositionId']==2])>=.75).sort_values(ascending=False)

In [ ]:
(df.loc[:,colsOASIS].isnull().sum()/len(df.loc[:,colsOASIS])).sort_values(ascending=True)

<hr style="height:2px;border-width:0;color:black;background-color:black">

<div class="alert alert-success" role="alert">
    <b>Create an events dataframe</b>
</div>

In [ ]:
cols_Oevents = ['M0100_ASSMT_REASON', 
                'M1860_CRNT_AMBLTN', 
                'M1850_CRNT_TRNSFRNG', 
                'M1830_CRNT_BATHG', 
                'M1820_CRNT_DRESS_LOWER', 
                'M1810_CRNT_DRESS_UPPER', 
                'M1306_UNHLD_STG2_PRSR_ULCR', 
                'M1242_PAIN_FREQ_ACTVTY_MVMT', 
                'M1400_WHEN_DYSPNEIC', 
                'M1340_SRGCL_WND_PRSNT', 
                'M1330_STAS_ULCR_PRSNT',
                'M1324_STG_PRBLM_ULCER', 
                'M1620_BWL_INCONT', 
                'M1800_CRNT_GROOMING',
                'M1030_THH_PAR_NUTRITION',
                'M1030_THH_NONE_ABOVE',
                'M1030_THH_IV_INFUSION',
                'M1630_OSTOMY',
                'M1030_THH_ENT_NUTRITION',
                'M1200_VISION',
                'M2200_THER_NEED_NA',
                'M1322_NBR_PRSULC_STG1']

In [ ]:
cols_Oevents = colsOASIS.tolist()

<div class="alert alert-success" role="alert">
    <b>Create an Items Mapping</b>
</div>

In [ ]:
event_mapping_df = pd.DataFrame(cols_Oevents,columns =['OASIS']).rename_axis('itemID').shift()[1:].reset_index(inplace=False)

In [ ]:
dtale.show(event_mapping_df)

In [ ]:
event_mapping_df.to_csv(csvStore+'/event_mapping_df.csv', sep=',', encoding='utf-8', index=False)

<hr style="height:2px;border-width:0;color:black;background-color:black">

<div class="alert alert-danger" role="alert">
    <b>Extract the integer from the event descriptions</b>
</div>

In [ ]:
df_events_int = df[cols_Oevents].apply(lambda x: x[cols_Oevents].str[:2].str.strip(), axis = 1).replace('NA',0)

In [ ]:
df_events_int.fillna(0, inplace=True)
df_events_int = df_events_int.replace('YE',2)
df_events_int = df_events_int.replace('NO',2)

In [ ]:
for column in df_events_int:
    if isinstance(df_events_int[column].dtypes, object):
        df_events_int[column]=df_events_int[column].astype(int)
        
#df_events_int = df_events_int.apply(np.int64)

In [ ]:
df_events_int.dtypes

In [ ]:
#df_events_desc = df[cols_Oevents].apply(lambda x: x[cols_Oevents].str[2:].str.strip(), axis = 1)

<div class="alert alert-danger" role="alert">
    <b>Combine the descriptive columns with the desired events</b>
</div>

In [ ]:
colsM = ['pa_id', 'epi_id', 'ceo_id', 'M0090_INFO_COMPLETED_DT']

In [ ]:
df_events = pd.concat([df[colsM], df_events_int], axis = 1).drop_duplicates().reset_index().drop('index', axis=1, errors='ignore')

In [ ]:
dtale.show(df_events.head())

<div class="alert alert-danger alert-info">
    <b>Convert from Wide to Long format</b>
</div>

In [ ]:
df_OASIS = df_events.melt(id_vars=colsM, var_name = 'OASIS', value_vars=cols_Oevents)

In [ ]:
df_OASIS = df_OASIS.merge(event_mapping_df,left_on='OASIS', right_on='OASIS', how='left')

<div class="alert alert-danger alert-info">
    <b>Convert itemID to Integers (this actually caused problems later on when serializing to JSON!!</b>
</div>

In [ ]:
df_OASIS.fillna({'itemID':0,}, inplace=True)
df_OASIS['itemID'] = df_OASIS['itemID'].apply(np.int64)

<div class="alert alert-danger alert-info">
    <b>Rearrange the columns to match MIMIC</b>
</div>

In [ ]:
mid = df_OASIS['itemID']
mid2 = df_OASIS['value']
df_OASIS.drop(labels=['itemID', 'value'], axis=1, inplace = True)
df_OASIS.insert(4, 'itemID', mid)
df_OASIS.insert(5, 'value', mid2)

In [ ]:
len(df_OASIS)
#df_OASIS.head()
dtale.show(df_OASIS.head())

<div class="alert alert-danger alert-info">
    <b>Export to CSV</b>
</div>

In [ ]:
df_OASIS.to_csv(csvStore+'/df_OASIS.csv', sep=',', encoding='utf-8', index=False)

<hr style="height:2px;border-width:0;color:black;background-color:black">

<div class="alert alert-success" role="alert">
  Create an visits df
</div>

In [ ]:
dfv = df.set_index(['pa_id', 'epi_id', 'ceo_id'])#[['M0090_INFO_COMPLETED_DT', 'M0100_ASSMT_REASON']]

In [ ]:
dtale.show(df[['M0090_INFO_COMPLETED_DT', 'M0100_ASSMT_REASON']].reset_index().drop_duplicates())

In [ ]:
df_visits = dfv[['M0090_INFO_COMPLETED_DT', 'M0100_ASSMT_REASON']].reset_index().drop('index', axis=1, errors='ignore').duplicated(keep=False)

In [ ]:
dtale.show(df[df_visits][['pa_id', 'epi_id', 'ceo_id','M0090_INFO_COMPLETED_DT', 'M0100_ASSMT_REASON']])

In [ ]:
df_visits = dfv[['M0090_INFO_COMPLETED_DT', 'M0100_ASSMT_REASON']].reset_index().drop('index', axis=1, errors='ignore').drop_duplicates().reset_index().drop('index', axis=1, errors='ignore')

In [ ]:
dtale.show(df_visits) #'epi_StartOfEpisode', 'epi_EndOfEpisode', 'epi_DischargeDate', 

In [ ]:
df_visits.to_csv(csvStore+'/df_visits.csv', sep=',', encoding='utf-8', index=False)

<hr style="height:2px;border-width:0;color:black;background-color:black">

<div class="alert alert-success" role="alert">
  Create an episodes df
</div>

In [ ]:
dfe = df.set_index(['pa_id', 'epi_id'])#[['M0090_INFO_COMPLETED_DT', 'M0100_ASSMT_REASON']]

In [ ]:
df_episodes = dfe[['mortality','epi_StartOfEpisode', 'epi_EndOfEpisode', 'epi_DischargeDate', 'epi_AdmitType',  'dcc_desc', 'dcr_desc', 'icd_code', 'icd_code_description']].reset_index().drop('index', axis=1, errors='ignore').drop_duplicates().reset_index().drop('index', axis=1, errors='ignore')

In [ ]:
int(~pd.isna(df_episodes.epi_DischargeDate).values[0])

In [ ]:
dtale.show(df_episodes)

In [ ]:
df_episodes.to_csv(csvStore+'/df_episode.csv', sep=',', encoding='utf-8', index=False)

<hr style="height:2px;border-width:0;color:black;background-color:black">